# Ayudantía 0000 1010: I/O y Serialización
### Autores: Pablo Araneda, Julio Huerta, Caua Santiago Paz


# Manejo de Bytes

Tu amigo te ha hablado sobre el NS Pro, también llamado como el Nuevo Sonido Pro. Tú como siempre, dudas un poco de estos rumores, pero tu amigo tiene pruebas y te manda un archivo de audio. Sin embargo, el audio está alterado y según él, este posee 10 veces más velocidad de modo que nadie lo pueda escuchar. Es aquí cuando con tus habilidades de computación te dispones a volver ese audio a la normalidad. Lo primero que haces es preguntarle a tu amigo si sabe qué tipo de archivo es. Él te dice que es un ".wav", un formato que también se llama "wave" y tú conoces muy bien.

Debido a lo anterior, te dispones a averiguar sobre este tipo de dato, y descubres la siguiente imágen:
![title](data/imgs/wav-sound-format.gif)
Además te encuentras información mas detallada en una pagina web: http://soundfile.sapp.org/doc/WaveFormat/

### lectura de archivos

Primero debes ver si coincide la información del archivo con lo que dice la tabla, para ver si lo estás haciendo bien, por lo que debes cargar el archivo. Extraemos solo los primeros 44 bytes (la metadata) porque el resto es solo el contenido del audio en sí.

In [ ]:
from os.path import join as op

with open(op("data", "dog.wav"), "rb") as file:  # abrimos en modo de lectura de bytes
    file_header_bytes = file.read(44)

Ahora podemos ver el contenido de estos bytes:

In [ ]:
print(file_header_bytes)
print("\n")
print(file_header_bytes[:4].decode("ASCII"))
print(file_header_bytes[12:16].decode("ASCII"))
print(file_header_bytes[36:40].decode("ASCII"))

Como se puede ver están los bytes en ASCII de "RIFF", "fmt ", y "data". Ahora podemos empezar a extraer los bytes y convertirlos

## Endianess (byteorder)
De la tabla del formato wave se tiene también que los números están en little endian, pero ¿Que significa esto?

El término inglés endianness ("extremidad") designa el formato en el que se almacenan los datos de más de un byte en un ordenador. El problema es similar a los idiomas en los que se escribe de derecha a izquierda, como el árabe, o el hebreo, frente a los que se escriben de izquierda a derecha.

![title](data/imgs/endianess.png)

## big-endian
Formato en el que el byte más significativo se almacena en primer lugar. Los demás bytes le siguen en orden de significado descendente. Por ejemplo el numero 123 sería representado como 123 y la letra "A" en formato unicode sería X'0041'

## little-endian
Formato en el que el byte menos significativo se almacena en primer lugar. Los demás bytes le siguen en orden de significado ascendente. Por ejemplo el numero 123 sería representado como 321. La letra "A" en formato unicode sería X'4100'

Volviendo a nuestro ejemplo, teníamos que los que los numeros de la tabla wave estan en formato little-endian. Por ejemplo, podemos ver la cantidad de canales (1 si es mono y 2 es estereo), que en la tabla está como "NumChannels". Esta se encuentra en el byte 22 y cubre 2 bytes, podemos imprimir estos valores:

In [ ]:
num_channels_bytes = file_header_bytes[22:22+2]
print(num_channels_bytes)

Los valores que se muestran están en hexadecimal (el "\x" indica eso). Y son dos hexadecimales, "02" y "00", debido a que el byteorder indicado en la tabla es little, el primer byte (el de más a la derecha) es el de menor valor.

## from_bytes
```int.from_bytes()``` nos permite convertir la información en bytes a un número. Con esto podemos pasar a ver la cantidad de canales, hay que recordar que deberemos de usar el byteorder little para leerlos en el orden correcto (ya que la tabla de wave dice que es así).

```int.from_bytes()``` recibe dos parámetros, los bytes a convertir y el orden en el que vienen

In [ ]:
num_channels_number = int.from_bytes(num_channels_bytes, byteorder="little")
print("canales:", num_channels_number)

Como se puede ver, se poseen dos canales, por lo que el audio es estereo. Podemos ver que pasa si le ponemos el byteorder incorrecto:

In [ ]:
num_channels_bad_number = int.from_bytes(num_channels_bytes, byteorder="big")
print("canales:", num_channels_bad_number)

Una vez reconocido el endianess, debes encargarte de convertir los valores necesarios para disminuir la frecuencia del sonido 10 veces, primero veamos la frecuencia que tiene el sonido.

En la tabla, la frecuencia se llama "SampleRate" (Ratio de muestreo), y está en el byte 24 y cubre 4 bytes

In [ ]:
frecuencia_bytes = file_header_bytes[24:24+4]
frecuencia = int.from_bytes(frecuencia_bytes, byteorder="little")
print("frecuencia:", frecuencia)

Ahora que tenemos la frecuencia, podemos ir a la tabla y ver qué valores se relacionan con esta. Es el ByteRate que describe el numero de bytes que se reproducen por sample, la formula es la siguiente:

ByteRate = SampleRate * NumChannels * BitsPerSample/8

Afortunadamente no es necesario realizar todo este calculo, podemos leer el archivo, obtener el valor del ByteRate y dividirlo por 10, ya que lo que estamos haciendo es unicamente disminuir el SampleRate de su fórmula. El ByteRate se encuentra en el byte 28, y cubre 4 bytes

In [ ]:
byte_rate_bytes = file_header_bytes[28:28+4]
byte_rate = int.from_bytes(byte_rate_bytes, byteorder="little")
print("ByteRate:", byte_rate)

## to_bytes

Ahora tenemos los valores, pero nos falta poder convertirlos a bytes de nuevo y escribirlos en el archivo. Primero podemos copiar el header y usarlo de base. Lo podemos convertir a un bytearray para poder trabajar de forma mas facil con sus datos

In [ ]:
nuevo_header_bytes = bytearray(file_header_bytes)
print(nuevo_header_bytes)

Debemos sobreescribir los bytes del ByteRate y la frecuencia, por ahora tenemos los valores como números, así que necesitamos convertirlos. Afortunadamente, existe la función ```int.to_bytes()```, que nos permite realizar esta conversion.

```int.to_bytes()``` recibe 2 parámetros que usaremos:
* La cantidad de bytes que usará el numero convertido
* El ```byteorder``` (endianess)

El valor del número se entrega en la parte de "int", por ejemplo:

In [ ]:
numero_a_convertir = 2233
numero_a_convertir.to_bytes(4, byteorder="big")

Se puede hacer esto en una línea, pero procurando usar un paréntesis alrededor del número:

In [ ]:
(2233).to_bytes(4, byteorder="big")

Ahora podemos pasar a convertir los valores de interes a bytes 

In [ ]:
# Frecuencia
nueva_frecuencia = frecuencia//10  # división entera
bytes_frecuencia = nueva_frecuencia.to_bytes(4, byteorder="little")
nuevo_header_bytes[24:24+4] = bytes_frecuencia

# Byte Rate
nuevo_byte_rate = byte_rate//10  # división entera
bytes_byte_rate = nuevo_byte_rate.to_bytes(4, byteorder="little")
nuevo_header_bytes[28:28+4] = bytes_byte_rate

## Escritura con bytes
Una vez actualizada la información, podemos finalmente escribir el nuevo archivo:

In [ ]:
file_in = open(op("data","dog.wav"), "rb")  # abrimos en modo de lectura de bytes
file_out = open(op("data","dog-slow.wav"), "wb")  # abrimos en modo de escritura de bytes
file_out.write(nuevo_header_bytes)

file_in.seek(44)  # avanzamos 44 bytes, ya que ya escribimos el header alterado

bytes_audio = file_in.read(4)  # vamos leyendo de a poco
while(len(bytes_audio)):
    file_out.write(bytes_audio)
    bytes_audio = file_in.read(4)  # vamos leyendo de a poco

# Cerramos los archivos    
file_in.close()
file_out.close()


# Serialización
¿Qué significa serializar? Consiste en un proceso de codificación de un objeto en un medio de almacenamiento

# Pickle vs JSON

![Resumen](data/imgs/resumen.png)

### Pickle
pickle.dump vs pickle.dumps, ¿cúal la diferencia?

```python
import pickle

```

```python
pickle.dumps(obj): Retorna la representación de pickle del objeto como bytes en el lugar de escribirla en un archivo.
```
```python
pickle.dump(obj, file): Escribe la representación del objeto como pickle en el archivo. 
```
fuente https://docs.python.org/3/library/pickle.html

Se puede deserializar con ```Pickle.loads``` y ```Pickle.load``` respectivamente.

### JSON

***json.load***s

```python
json.load(file, object_hook=None **kw)
```

***Object_hook***

En pickle ocupamos el __\_\_setstate\_\___ y __\_\_getstate\_\___ para personalizar nuestra serializacion, ya en json __object_hook__ y __\_\_getstate\_\___

La unica diferencia es que para hacer el __\_\_getstate\_\___ de json hay que heredar de json.JSONEncoder y sobreescribir el método default

***Con ese mini repaso, ya estan listos para el ejercicio***

### Contexto
Parte Bytes: En esta parte deberás reparar un archivo cuyos bytes fueron manipulados, con el fin
de obtener un archivo de imagen con los datos que necesitas del ayudante jefe coordinador Enzo
Tamburini. Aplicando tus conocimientos de manipulación de bytes encontrarás el usuario y clave
necesarios para conocer a Enzo.

Parte Pickle: En esta parte deberás extraer información de los ayudantes de la sección de Docencia, donde se encuentran los datos que necesitas del ayudante jefe de docencia Dante Pinto.
Aplicando tus conocimientos de serialización mediante pickle encontrarás el usuario y clave necesario para conocer a Dr. Pinto Dante.

### Parte Pickle

In [ ]:
import pickle

```python
def __setstate__(self, estado): 
```
En este método debes personalizar la deserialización
que hará pickle. Primero debes recordar que el argumento estado de este método es un
diccionario cuyas llaves son los nombres de cada atributo y el contenido es el valor del
atributo correspondiente.
En este método se debe encontrar una instancia de AyudanteJefe al interior de la lista
de ayudantes (almacenada bajo la llave ayudantes_normales). Para encontrarla, debes
recordar que esta lista contiene instancias de la clase Ayudante y AyudanteJefe y ambas
cuentan con un atributo llamado cargo que será igual a "Jefe" sólo si la instancia es de
la clase AyudanteJefe.
Una vez encontrado el AyudanteJefe, debes sacar la instancia de la lista y asignarlo al
atributo ayudante_jefe de forma que la lista final en ayudantes_normales contenga las
instancias de todos los ayudantes que no son jefes.


In [ ]:
# ETAPA DE CARGA #
class EquipoDocencia:
    def __init__(self):
        self.ayudantes_normales = []
        self.ayudante_jefe = None

    def __setstate__(self, estado):  # acá se filtra la lista del objeto al deserializarlo
        pass

```python
• def cargar_instancia(ruta):
```
Recibe la ruta que corresponde a la ubicación de un archivo a cargar, y retorna el archivo cargado usando pickle

In [ ]:
def cargar_instancia(ruta):  # Se carga la instancia de EquipoDocencia
    pass

In [ ]:
# ETAPA DE GUARDADO #
class Ayudante:
    def __init__(self, cargo, usuario_github, pokemon_favorito, pizza_favorita):
        self.cargo = cargo
        self.usuario_github = usuario_github
        self.pokemon_favorito = pokemon_favorito
        self.pizza_favorita = pizza_favorita

    def __repr__(self):
        mensaje = f"¡Hola! soy {self.usuario_github} y tengo el cargo de {self.cargo}"
        return mensaje

```python
• class AyudanteJefe(Ayudante):
```
Corresponde al ayudante jefe de docencia. Hereda de
Ayudante. Debes modificar uno de sus métodos.

```python
◦ def __getstate__(self):
```
En este método debes personalizar la serialización, modificando el estado de los atributos que se guardarán de la clase. Específicamente, debes asegurarte
de que pizza_favorita sea None, trabajo_restante sea igual a "Nada" y experto sea
igual a "TortugaNinja".

In [ ]:
class AyudanteJefe(Ayudante):
    def __init__(self, cargo, usuario_github, pokemon_favorito, pizza_favorita, trabajo_restante, experto, carrera):
        super().__init__(cargo, usuario_github, pokemon_favorito, pizza_favorita)
        self.trabajo_restante = trabajo_restante
        self.experto = experto
        self.carrera = carrera

    def __getstate__(self):  # cambios que sólo se hacen a AyudanteJefe
        pass

```python
• def guardar_instancia(ruta, objeto_lista_ayudantes):
```
Recibe la ruta que corresponde a la ubicación de un archivo vacío llamado equipo_corregido.bin y una instancia de
EquipoDocencia. La función debe serializar la instancia en la ruta de el archivo nuevo, mediante
pickle, y luego retornar True.

In [ ]:
def guardar_instancia(ruta, instancia_equipo_docencia):  # Se guarda instancia de EquipoDocencia
    pass

### Parte Bytes

La información del ayudante jefe coordinador se encuentra corrompida en un archivo cuyos bytes fueron
manipulados. Tendrás que reparar dicho archivo para recuperar los datos del ayudante, completando la
función reparar_usuario que se encuentra dentro del archivo reparar_bytes.py.
def reparar_usuario(ruta): Esta función recibe la ruta del archivo a reparar, lee el archivo
como bytes y mediante un algoritmo los modifica para escribir los bytes originales en el archivo
user_info.bmp. El algoritmo recorre segmentos de 32 bytes contiguos y para cada uno sabes
que:

1. El primer byte corresponde a un entero que puede ser 1 o 0, los siguientes 16 bytes pertenecen
al archivo original y los últimos 15 NO pertenecen al archivo original.
2. Si el primer byte es un 1, significa que los siguientes 16 bytes han sido invertidos, mientras que
si es un 0, entonces están en el orden original. Por invertido, se refiere a que si la secuencia es
1 2 3 4, entonces la original es 4 3 2 1.

3. Es necesario extraer los bytes originales al archivo en el orden que corresponda. Luego se avanza
al siguiente segmento de 32 bytes.

Si se concatenan las porciones de bytes extraídas, respetando el orden de los segmentos de 32 bytes, se
obtiene el contenido completo que se escribe en el archivo de imagen user_info.bmp. Al abrir la imagen,
si se aplicó el algoritmo correctamente, se revelarán los datos del ayudante jefe.

In [ ]:
def reparar_imagen(ruta):
    # Completar esta función
    pass

if __name__ == '__main__':
    try:
        reparar_imagen(op('data', 'Parte Bytes', 'imagen_danada.xyz'))
        print("Contraseña reparada")
    except Exception as error:
        print(f'Error: {error}')
        print("No has podido obtener la información del Ayudante!")